# 4.6 IC Combining and Exporting Data 01

## Contents:

### 01. Import the data sets into Jupyter

### 02. Check the dimensions of the imported dataframes

### 03. Merge the dataframes

### 04. Determine if there is a full match between two dataframes

### 05. Export data in pickle format

In [2]:
# Import libraries
import pandas as pd
import numpy as np
import os

#### 01. Import the data sets into Jupyter

In [3]:
# Importing df_ords_prior
path = r'/Users/yaseminmustafa/Desktop/CareerFoundry/Exercise 4/15-05-2025_Instacart Basket Analysis'

In [4]:
path

'/Users/yaseminmustafa/Desktop/CareerFoundry/Exercise 4/15-05-2025_Instacart Basket Analysis'

In [5]:
df_ords_prior = pd.read_csv(os.path.join(path,"02_Data/Original Data/orders_products_prior.csv"), index_col = False)

In [6]:
# Check the output
df_ords_prior.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [8]:
# Importing df_ords - df_orders_checked
df_ords = pd.read_csv(os.path.join(path,"02_Data/Prepared Data/orders_checked.csv"), index_col = False)

In [9]:
# Drop unnecessary columns
df_ords = df_ords.drop(columns=['Unnamed: 0', 'Unnamed: 0.1'])

In [10]:
df_ords.head()

,order_id,user_id,dataset_label,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order
0,2398795,1,prior,2,3,7,15.0
1,473747,1,prior,3,3,12,21.0
2,2254736,1,prior,4,4,7,29.0
3,431534,1,prior,5,4,15,28.0
4,3367565,1,prior,6,2,7,19.0


#### 02. Check the dimensions of the imported dataframes

In [11]:
# Checking dimensions of df_ords_prior
df_ords_prior.shape

(32434489, 4)

In [12]:
# Checking dimensions of df_ords
df_ords.shape

(3214874, 7)

#### 03. Merge the dataframes

In [13]:
# Merging dataframes
df_merged_large = df_ords.merge(df_ords_prior, on = 'order_id', indicator = True)

In [14]:
# Check output
df_merged_large.head()

,order_id,user_id,dataset_label,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,_merge
0,2398795,1,prior,2,3,7,15.0,196,1,1,both
1,2398795,1,prior,2,3,7,15.0,10258,2,0,both
2,2398795,1,prior,2,3,7,15.0,12427,3,1,both
3,2398795,1,prior,2,3,7,15.0,13176,4,0,both
4,2398795,1,prior,2,3,7,15.0,26088,5,1,both


In [15]:
# Checking the shape of df_merged_large
df_merged_large.shape

(30356421, 11)

In [16]:
# Frequency check
df_merged_large['_merge'].value_counts()

_merge
both          30356421
left_only            0
right_only           0
Name: count, dtype: int64

In [17]:
df_merged_large = df_ords.merge(df_ords_prior, on = 'order_id', indicator = True, how = 'outer')

In [18]:
# Check output
df_merged_large.head()

,order_id,user_id,dataset_label,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,_merge
0,1,112108.0,train,4.0,4.0,10.0,9.0,NaN,NaN,NaN,left_only
1,2,202279.0,prior,3.0,5.0,9.0,8.0,33120.0,1.0,1.0,both
2,2,202279.0,prior,3.0,5.0,9.0,8.0,28985.0,2.0,1.0,both
3,2,202279.0,prior,3.0,5.0,9.0,8.0,9327.0,3.0,0.0,both
4,2,202279.0,prior,3.0,5.0,9.0,8.0,45918.0,4.0,1.0,both


In [19]:
# Checking the shape of df_merged_large
df_merged_large.shape

(32640698, 11)

#### 04. Determine if there is a full match between two dataframes

In [20]:
# Frequency check
df_merged_large['_merge'].value_counts()

_merge
both          30356421
right_only     2078068
left_only       206209
Name: count, dtype: int64

#### 05. Export data in pickle format

In [21]:
# Export data to pkl
df_merged_large.to_pickle(os.path.join(path, '02_Data','Prepared Data', 'orders_products_combined.pkl'))